In [1]:
import os
from pathlib import Path
import glob
import re
import numpy as np
import pandas as pd
import h5py
import torch
#import librosa
import ast
import string
import zipfile
from tqdm.notebook import tqdm
from sklearn.linear_model import RidgeCV, Ridge
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.stats import pearsonr
import cv2
from moviepy.editor import VideoFileClip
import nibabel as nib
from nilearn import plotting
from nilearn.maskers import NiftiLabelsMasker
import ipywidgets as widgets
from ipywidgets import VBox, Dropdown, Button
from IPython.display import Video, display, clear_output
from transformers import BertTokenizer, BertModel
from torchvision.transforms import Compose, Lambda, CenterCrop
from torchvision.models.feature_extraction import create_feature_extractor
from pytorchvideo.transforms import Normalize, UniformTemporalSubsample, ShortSideScale
# Load the .mkv file
import zipfile

In [2]:
root_data_dir = './data/algonauts_2025.competitors'
# Select platform
platform = 'jupyter_notebook' #@param ['colab', 'jupyter_notebook']
initial_dir = os.getcwd() 
# Select device for computation
device = 'cpu' # @param ['cpu', 'cuda']

print(f'Running on "{platform}" using "{device}" device!')

Running on "jupyter_notebook" using "cpu" device!


In [3]:
# Open the fMRI responses file, and extract the specific dataset 
fmri_file_path = root_data_dir + "/fmri/sub-01/func/sub-01_task-friends_space-MNI152NLin2009cAsym_atlas-Schaefer18_parcel-1000Par7Net_desc-s123456_bold.h5"

# Open the H5 file in read mode
with h5py.File(fmri_file_path, 'r') as file:
    print(f"Keys: {list(file.keys())}")
    a_group_key = list(file.keys())
    for i in a_group_key:  
        if "s01e02a" in i:
            print("Found: ", i)
            dataset_name = i
            break
    fmri_data = file[dataset_name][()]



Keys: ['ses-001_task-s01e02a', 'ses-001_task-s01e02b', 'ses-001_task-s01e03a', 'ses-001_task-s01e03b', 'ses-002_task-s01e04a', 'ses-002_task-s01e04b', 'ses-002_task-s01e05a', 'ses-002_task-s01e05b', 'ses-003_task-s01e01a', 'ses-003_task-s01e01b', 'ses-003_task-s01e06a', 'ses-003_task-s01e06b', 'ses-004_task-s01e07a', 'ses-004_task-s01e07b', 'ses-004_task-s01e08a', 'ses-004_task-s01e08b', 'ses-004_task-s01e09a', 'ses-004_task-s01e09b', 'ses-005_task-s01e10a', 'ses-005_task-s01e10b', 'ses-005_task-s01e11a', 'ses-005_task-s01e11b', 'ses-006_task-s01e12a', 'ses-006_task-s01e12b', 'ses-006_task-s01e13a', 'ses-006_task-s01e13b', 'ses-006_task-s01e14a', 'ses-006_task-s01e14b', 'ses-007_task-s01e15a', 'ses-007_task-s01e15b', 'ses-007_task-s01e16a', 'ses-007_task-s01e16b', 'ses-007_task-s01e17a', 'ses-007_task-s01e17b', 'ses-008_task-s01e18a', 'ses-008_task-s01e18b', 'ses-008_task-s01e19a', 'ses-008_task-s01e19b', 'ses-009_task-s01e20a', 'ses-009_task-s01e20b', 'ses-009_task-s01e21a', 'ses-009_

# audio features 

In [ ]:
import torchaudio
from pathlib import Path

bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
model = bundle.get_model().to(device)

def extract_audio_features(episode_path, tr,  device, save_dir_temp,
    save_dir_features):
    name_ep=episode_path.split('/')[-1].split('_')[-1].split('.')[0]
    # Get the onset time of each movie chunk
    clip = VideoFileClip(episode_path)
    start_times = [x for x in np.arange(0, clip.duration, tr)][:-1]
    # Create the directory where the movie chunks are temporarily saved
    temp_dir = os.path.join(save_dir_temp, 'temp')
    os.makedirs(temp_dir, exist_ok=True)
    # Empty features list
    audio_features = []
    ### Loop over chunks ###
    with tqdm(total=len(start_times), desc="Extracting audio features") as pbar:
        for start in start_times:

            clip_chunk = clip.subclip(start, start+tr)
            chunk_audio_path = os.path.join(temp_dir, 'audio_s01e01a.wav')
            clip_chunk.audio.write_audiofile(chunk_audio_path, verbose=False,
                            logger=None)
            waveform, sample_rate = torchaudio.load(chunk_audio_path) 
            waveform = torch.mean(waveform, 0,True) # average the two channels of the wave files
            waveform = waveform.to(device)
            if sample_rate != bundle.sample_rate:
                waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)
            with torch.inference_mode():
                features, _ = model.extract_features(waveform)
                features = np.array(features, dtype='float32')
                # keep last layer() of the model
                features = features[11,:,:,:]
                features = np.mean(features,1) # average the time frames 
                audio_features.append(features)
 
                # Update the progress bar
                pbar.update(1)
    ### Convert the visual features to float32 ###
    audio_features = np.array(audio_features, dtype='float32')
    # Save the audio features
    out_file_audio = os.path.join(
        save_dir_features,name_ep+'_features_audio.h5')
    print(f"Saving audio features to {out_file_audio}")
    with h5py.File(out_file_audio, 'a' if Path(out_file_audio).exists() else 'w') as f:
        group = f.create_group(name_ep)
        group.create_dataset('audio', data=audio_features, dtype=np.float32)
    print(f"Audio features saved to {out_file_audio}")


objc[79333]: Class AVFFrameReceiver is implemented in both /opt/anaconda3/envs/py39/lib/python3.9/site-packages/av/.dylibs/libavdevice.61.3.100.dylib (0x3184d43a8) and /opt/anaconda3/envs/py39/lib/libavdevice.59.7.100.dylib (0x3215d8778). One of the two will be used. Which one is undefined.
objc[79333]: Class AVFAudioReceiver is implemented in both /opt/anaconda3/envs/py39/lib/python3.9/site-packages/av/.dylibs/libavdevice.61.3.100.dylib (0x3184d43f8) and /opt/anaconda3/envs/py39/lib/libavdevice.59.7.100.dylib (0x3215d87c8). One of the two will be used. Which one is undefined.


In [4]:
def get_ep_path(documents_path):
    os.chdir("../algonauts_2025.competitors")
    paths=[glob.glob(os.path.join(documents_path, "*.mkv"))]
    return paths

In [ ]:
# As an example, extract audio features using season 1, episode 1 of Friends

show_path =["/stimuli/movies/friends/","/stimuli/movies/movie10"]
documents_path = os.getcwd() +"/stimuli/movies/friends/s1"
episode_paths = get_ep_path(documents_path)
# Duration of each movie chunk, aligned with the fMRI TR of 1.49 seconds
tr = 1.49
# Saving directories
save_dir_temp = initial_dir+"/data/audio_features/"
save_dir_features = initial_dir+"/data/audio_features/friends_features/"
print(f"Saving audio features to {save_dir_features}")

# Execute audio feature extraction
for i in episode_paths[0]:
    extract_audio_features(i, tr, device,
        save_dir_temp, save_dir_features)

Saving audio features to /Users/genevievelam/Documents/GitHub/algonauts_2025_challenge/data/audio_features/friends_features/
Processing /Users/genevievelam/Documents/GitHub/algonauts_2025.competitors/stimuli/movies/friends/s1/friends_s01e09b.mkv


Extracting audio features:   0%|          | 0/467 [00:00<?, ?it/s]

s01e09b
Saving audio features to /Users/genevievelam/Documents/GitHub/algonauts_2025_challenge/data/audio_features/friends_features/s01e09b_features_audio.h5
Audio features saved to /Users/genevievelam/Documents/GitHub/algonauts_2025_challenge/data/audio_features/friends_features/s01e09b_features_audio.h5


In [7]:
def define_frames_transform():
    """Defines the preprocessing pipeline for the video frames. Note that this
    transform is specific to the slow_r50 model."""
    transform = Compose(
        [
            UniformTemporalSubsample(8),
            Lambda(lambda x: x/255.0),
            Normalize([0.45, 0.45, 0.45], [0.225, 0.225, 0.225]),
            ShortSideScale(size=256),
            CenterCrop(256)
        ]
  )
    return transform

transform = define_frames_transform()

In [8]:
def get_vision_model(device):
    """
    Load a pre-trained slow_r50 video model and set up the feature extractor.

    Parameters
    ----------
    device : torch.device
        The device on which the model will run (i.e., 'cpu' or 'cuda').

    Returns
    -------
    feature_extractor : torch.nn.Module
        The feature extractor model.
    model_layer : str
        The layer from which visual features will be extracted.

    """

    # Load the model
    model = torch.hub.load('facebookresearch/pytorchvideo', 'slow_r50',
        pretrained=True)

    # Select 'blocks.5.pool' as the feature extractor layer
    model_layer = 'blocks.5.pool'
    feature_extractor = create_feature_extractor(model,
        return_nodes=[model_layer])
    feature_extractor.to(device)
    feature_extractor.eval()

    return feature_extractor, model_layer

feature_extractor, model_layer = get_vision_model(device)

Using cache found in /Users/genevievelam/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [ ]:
def extract_visual_features(episode_path, tr, feature_extractor, model_layer,
    transform, device, save_dir_temp, save_dir_features):
    """
    Extract visual features from a movie using a pre-trained video model.

    Parameters
    ----------
    episode_path : str
        Path to the movie file for which the visual features are extracted.
    tr : float
        Duration of each chunk, in seconds (aligned with the fMRI repetition
        time, or TR).
    feature_extractor : torch.nn.Module
        Pre-trained feature extractor model.
    model_layer : str
        The model layer from which the visual features are extracted.
    transform : torchvision.transforms.Compose
        Transformation pipeline for processing video frames.
    device : torch.device
        Device for computation ('cpu' or 'cuda').
    save_dir_temp : str
        Directory where the chunked movie clips are temporarily stored for
        feature extraction.
    save_dir_features : str
        Directory where the extracted visual features are saved.

    Returns
    -------
    visual_features : float
        Array containing the extracted visual features.

    """

    # Get the onset time of each movie chunk
    name_ep=episode_path.split('/')[-1].split('_')[-1].split('.')[0]
    clip = VideoFileClip(episode_path)
    start_times = [x for x in np.arange(0, clip.duration, tr)][:-1]
    # Create the directory where the movie chunks are temporarily saved
    temp_dir = os.path.join(save_dir_temp, 'temp')
    os.makedirs(temp_dir, exist_ok=True)

    # Empty features list
    visual_features = []

    # Loop over chunks
    with tqdm(total=len(start_times), desc="Extracting visual features") as pbar:
        for start in start_times:

            # Divide the movie in chunks of length TR, and save the resulting
            # clips as '.mp4' files
            clip_chunk = clip.subclip(start, start+tr)
            chunk_path = os.path.join(temp_dir, 'visual_chunk.mp4')
            clip_chunk.write_videofile(chunk_path, verbose=False, audio=False,
                logger=None)
            # Load the frames from the chunked movie clip
            video_clip = VideoFileClip(chunk_path)
            chunk_frames = [frame for frame in video_clip.iter_frames()]

            # Format the frames to shape:
            # (batch_size, channels, num_frames, height, width)
            frames_array = np.transpose(np.array(chunk_frames), (3, 0, 1, 2))
            # Convert the video frames to tensor
            inputs = torch.from_numpy(frames_array).float()
            # Preprocess the video frames
            inputs = transform(inputs).unsqueeze(0).to(device)

            # Extract the visual features
            with torch.no_grad():
                preds = feature_extractor(inputs)
            visual_features.append(np.reshape(preds[model_layer].cpu().numpy(), -1))

            # Update the progress bar
            pbar.update(1)

    # Convert the visual features to float32
    visual_features = np.array(visual_features, dtype='float32')

    # Save the visual features
    out_file_visual = os.path.join(
        save_dir_features, name_ep+'_features_visual.h5')
    with h5py.File(out_file_visual, 'a' if Path(out_file_visual).exists() else 'w') as f:
        group = f.create_group(name_ep)
        group.create_dataset('visual', data=visual_features, dtype=np.float32)
    print(f"Visual features saved to {out_file_visual}")

    # Output
    return visual_features

In [ ]:
# As an exemple, extract visual features for season 1, episode 1 of Friends
documents_path = os.getcwd() +"/stimuli/movies/friends/s1"
episode_paths = get_ep_path(documents_path)
# Duration of each movie chunk, aligned with the fMRI TR of 1.49 seconds
tr = 1.49
# Saving directories
save_dir_temp = initial_dir+"/data/visual_features/"
save_dir_features = initial_dir+"/data/visual_features/friends_features/"
print(f"Saving audio features to {save_dir_features}")

# Execute audio feature extraction
for i in episode_paths[0][1:2]:
    print(f"Processing {i}")
    extract_visual_features(i, tr, feature_extractor,
    model_layer, transform, device, save_dir_temp, save_dir_features)